In [1]:
import os
import pandas as pd
import numpy as np
import json


#pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

## Overview

The Houston Rockets collect a wide range of data including ticket transactions, retail sales, and fan surveys. However, this data comes from various sources with differing formats, making it difficult to truly understand who our fans are and track how they’re interacting with the Houston Rockets.

Use the available data sources to create a unified database table (i.e., a single table that our Business Intelligence & Innovation team could leverage to build fan segments and determine their behaviors).

**Requirements**

Using your programming method of choice, create a unified database table that could be used as the basis for dashboards and reporting on fan segments and their behaviors.

At minimum, the database table should include the following:

- A unique identifier for each fan as the primary key
- Fan identifiers from each data source
- Fields containing contact information for each fan (email, phone number, and zip code)

The following calculated fields:
- Number of ticket transactions
- Number of retail transactions
- Number of survey responses

At least four additional calculated fields. For example:
- Average ticket price for each fan
- Fan total spend


Project is available on Github, or similar SVN service, with a README on how to locally view and/or run your project. If a private repo, which we would encourage, please add @mkamla as a collaborator when your project is ready for review.
Timely completion of the project. Preferably no more than 7 days from delivery of project details.

**Evaluation Criteria**

Aside from adherence to the requirements, below are specific aspects that will be evaluated:

- Inclusion of supporting files, documentation and scripts used to generate the unified table
- Thoughtful consideration to datapoints that are relevant to a sports, entertainment and/or event business

**Bonus Points**
Additional consideration will be given to projects that include details about your methodology or approach, insights uncovered, supplemental tables and creativity in incorporating external resources that are additive to the project requirements and may reside outside the scope of this document.

The difference between ordinary and extraordinary is a little extra.



## Acquistion

In [5]:
# Import JSON file
json_data = pd.read_json('retail.json', orient='columns')

# Import CSV files
survey_data = pd.read_csv('surveys.csv')
ticket_data = pd.read_csv('tickets.csv')


# Print the first few rows of each data frame

print("JSON - retail data:")
print(json_data.head())

print("CSV - survey data:")
print(survey_data.head())

print("CSV - ticket data :")
print(ticket_data.head())

JSON - retail data:
                                              retail
0  {'transaction_id': 1, 'email': 'user18@rockets...
1  {'transaction_id': 2, 'email': 'user142@rocket...
2  {'transaction_id': 3, 'email': 'user182@rocket...
3  {'transaction_id': 4, 'email': 'user492@rocket...
4  {'transaction_id': 5, 'email': 'user101@rocket...
CSV - survey data:
   Submission ID                                          Attribute            Value
0              1                                           phone_no     290-551-1299
1              1                                           event_id             3220
2              1             how_satisfied_were_you_with_this_event                2
3              1  how_satisfied_were_you_with_your_retail_experi...                3
4              1  how_likely_are_you_to_attend_this_event_in_the...  5 - Very Likely
CSV - ticket data :
   transaction_id  account_no                email    zip      phone_no  section  row  qty  total_price  event_id

In [4]:
print("Info - retail data :")
json_data.info()

print("Info - survey data :")
survey_data.info()

print("Info - ticket data :")
ticket_data.info()

Info - retail data :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   retail  2000 non-null   object
dtypes: object(1)
memory usage: 15.8+ KB
Info - survey data :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Submission ID  12000 non-null  int64 
 1   Attribute      12000 non-null  object
 2   Value          12000 non-null  object
dtypes: int64(1), object(2)
memory usage: 281.4+ KB
Info - ticket data :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   transaction_id  10000 non-null  int64 
 1   account_no      10000 non-null  object
 2   email           10

**Takeaways**

- Retail JSON 

    - No nulls.
    - Currently 2000 rows and 1 column.

     **Things to Do**

    - Need to convert to Dataframe with the following fields:
        - transaction_id, email, account_no , product_type, quantity, unit_price , shipping cost. 
        - Dataframe will have 2000 rows with 7 columns.
        

- Ticket data

    - No nulls
    - Dataframe is 10,000 rows and 11 columns

- Survey data

    - No nulls
    - Currently 12,000 rows and 3 columns.

    **Things to Do**
    
    - Need to pivot data on 'Submission ID' index and 'Attribute' being columns. Final shape will be determine after transforming the data table.
    - normalize some data fields

**MVP**

- Master Fan Dataframe

    - need Unique ID for each fan as primary key
    - ID for each data table 
    - Contact information for each fan
        - email
        - phone number
        - zip code
    - Fields calculating 
        - Number of ticket transactions
        - Number of retail transactions
        - Number of surveys completed
    - Additional fields (Need 4)
        - Avg ticket price per fan
        - Fan overall total (ticket + retail)
        - Mode of seating (section) attendence (Maybe) per fan
        - last additional field to be determine.



## Prepare the data

### Retail Data

In [7]:
# Read JSON data from file
with open('retail.json') as f:
    json_data = json.load(f)

# Extract data from JSON key 'retail'
data = json_data["retail"]

# Create DataFrame from JSON data
df = pd.DataFrame(data, columns=["transaction_id",
                                 "email",
                                 "account_no",
                                 "product_type",
                                 "quantity",
                                 "unit_price",
                                 "shipping_cost"])

df.head()


,transaction_id,email,account_no,product_type,quantity,unit_price,shipping_cost
0,1,user18@rockets.com,E894194JJ481,Jersey,2,96,5.76
1,2,user142@rockets.com,G684186GK636,Misc,5,9,1.35
2,3,user182@rockets.com,X898402TO472,Jersey,3,98,8.82
3,4,user492@rockets.com,R226999ZA574,Jersey,4,104,12.48
4,5,user101@rockets.com,Q640255YC818,Jersey,3,98,8.82


In [8]:
# Add new column with transaction total calculation
df["transaction_total"] = df["quantity"] * df["unit_price"] + df["shipping_cost"]

df

,transaction_id,email,account_no,product_type,quantity,unit_price,shipping_cost,transaction_total
0,1,user18@rockets.com,E894194JJ481,Jersey,2,96,5.76,197.76
1,2,user142@rockets.com,G684186GK636,Misc,5,9,1.35,46.35
2,3,user182@rockets.com,X898402TO472,Jersey,3,98,8.82,302.82
3,4,user492@rockets.com,R226999ZA574,Jersey,4,104,12.48,428.48
4,5,user101@rockets.com,Q640255YC818,Jersey,3,98,8.82,302.82
...,...,...,...,...,...,...,...,...
1995,1996,user88@rockets.com,H383584PU325,Hat,2,24,1.44,49.44
1996,1997,user410@rockets.com,M618220JQ428,Misc,6,7,1.26,43.26
1997,1998,user326@rockets.com,L452536ZY996,Jersey,2,104,6.24,214.24
1998,1999,user193@rockets.com,U673743FK544,Misc,6,9,1.62,55.62


**Takeaways**

- Converted the JSON object to an apriopate dataframe by extracting data from the 'retail' key.
- added new calculation column with the sum of transaction adding subtotal (quanitity * unit_price) and shipping cost.

### Ticket Data

In [13]:
ticket_data.head()

,transaction_id,account_no,email,zip,phone_no,section,row,qty,total_price,event_id,channel
0,1,A87144476G,user400@rockets.com,77066,280-379-5220,109,9,1,200,3223,Web
1,2,A66578188Z,user141@rockets.com,76673,490-491-8071,101,10,4,800,3221,Box Office
2,3,A11689958W,user98@rockets.com,77031,244-805-9413,100,18,8,1600,3237,Box Office
3,4,A47432461Z,user213@rockets.com,76136,826-458-9773,400,7,1,50,3240,Web
4,5,A80089942I,user472@rockets.com,75559,803-733-6051,414,17,1,25,3215,Box Office


### Survey data 

In [10]:
survey_data.head(20)

,Submission ID,Attribute,Value
0,1,phone_no,290-551-1299
1,1,event_id,3220
2,1,how_satisfied_were_you_with_this_event,2
3,1,how_satisfied_were_you_with_your_retail_experi...,3
4,1,how_likely_are_you_to_attend_this_event_in_the...,5 - Very Likely
5,1,what_is_your_birthdate,33939
6,1,what_is_your_household_income,"Less than $50,000"
7,1,what_is_your_highest_level_of_education_that_y...,Associate's Degree
8,2,phone_no,663-795-4865
9,2,event_id,3242


In [12]:
survey_data = survey_data.pivot(index= 'Submission ID', columns= 'Attribute')

Value                                                                                                                                                                                                                                                                                                
Attribute     event_id how_likely_are_you_to_attend_this_event_in_the_future how_satisfied_were_you_with_this_event how_satisfied_were_you_with_your_retail_experience_at_this_event      phone_no what_is_your_birthdate what_is_your_highest_level_of_education_that_you_have_attained what_is_your_household_income
Submission ID                                                                                                                                                                                                                                                                                                         
1                 3220                                    5 - Very Likely                                         2                                                  3                290-551-1299                  33939                                 Associate's Degree                         Less than $50,000
2                 3242                                  1 - Very Unlikely                        5 - Very Satisfied                                                  3                663-795-4865                  21535                                  Vocational School                         Less than $50,000
3                 3217                                                  3                                         4                                                  2                674-251-1148                  35693                                    Graduate Degree                       $100,000 - $149,000
4                 3215                                                  4                                         3                                                  3                728-127-6014                  37384                                  Vocational School                         Less than $50,000
5                 3237                                                  3                        5 - Very Satisfied                                                  3                238-199-2712                  22531                                  Vocational School                           $250,00 or more
...                ...                                                ...                                       ...                                                ...                         ...                    ...                                                ...                                       ...
1496              3223                                                  2                        5 - Very Satisfied                                                  4                610-160-7313                  18723                                  Bachelor's Degree                         $50,000 - $99,999
1497              3246                                  1 - Very Unlikely                                         3                                                  3                464-208-9383                  27239                                  Vocational School                           $250,00 or more
1498              3221                                                  4                                         4                                                  3                534-779-8923                  17259                                  Bachelor's Degree                       $200,000 - $249,000
1499              3249                                  1 - Very Unlikely                                         4                                                  3                891-415-5232                  22432                                    Graduate Degree                 

In [11]:
survey_data.pivot(index= 'Submission ID', columns= 'Attribute').info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1500 entries, 1 to 1500
Data columns (total 8 columns):
 #   Column                                                                     Non-Null Count  Dtype 
---  ------                                                                     --------------  ----- 
 0   (Value, event_id)                                                          1500 non-null   object
 1   (Value, how_likely_are_you_to_attend_this_event_in_the_future)             1500 non-null   object
 2   (Value, how_satisfied_were_you_with_this_event)                            1500 non-null   object
 3   (Value, how_satisfied_were_you_with_your_retail_experience_at_this_event)  1500 non-null   object
 4   (Value, phone_no)                                                          1500 non-null   object
 5   (Value, what_is_your_birthdate)                                            1500 non-null   object
 6   (Value, what_is_your_highest_level_of_education_that_you_have_att

### Create Acquire function

In [14]:
def import_data(json_file, csv_file1, csv_file2):
    # Import JSON file into a dataframe
    with open(json_file) as f:
        json_data = json.load(f)
    retail_data = pd.DataFrame(json_data["retail"], columns=["transaction_id", "email", "account_no", "product_type", "quantity", "unit_price", "shipping_cost"])

    # Import CSV files into dataframes
    survey_data = pd.read_csv(csv_file1)
    ticket_data = pd.read_csv(csv_file2)
    
    return retail_data, survey_data, ticket_data

In this code, the import_data function takes three file paths as input: json_file, csv_file1, and csv_file2. It reads the JSON file and creates a DataFrame retail_data from the specified JSON data. It also imports the two CSV files and creates separate DataFrames survey_data and ticket_data. Finally, the function returns these three dataframes.

In [19]:
# Usage of function
json_file = 'retail.json'
csv_file1 = 'surveys.csv'
csv_file2 = 'tickets.csv'

retail_data, survey_data, ticket_data = import_data(json_file, csv_file1, csv_file2)

In [16]:
retail_data

,transaction_id,email,account_no,product_type,quantity,unit_price,shipping_cost
0,1,user18@rockets.com,E894194JJ481,Jersey,2,96,5.76
1,2,user142@rockets.com,G684186GK636,Misc,5,9,1.35
2,3,user182@rockets.com,X898402TO472,Jersey,3,98,8.82
3,4,user492@rockets.com,R226999ZA574,Jersey,4,104,12.48
4,5,user101@rockets.com,Q640255YC818,Jersey,3,98,8.82
...,...,...,...,...,...,...,...
1995,1996,user88@rockets.com,H383584PU325,Hat,2,24,1.44
1996,1997,user410@rockets.com,M618220JQ428,Misc,6,7,1.26
1997,1998,user326@rockets.com,L452536ZY996,Jersey,2,104,6.24
1998,1999,user193@rockets.com,U673743FK544,Misc,6,9,1.62


In [17]:
ticket_data

,transaction_id,account_no,email,zip,phone_no,section,row,qty,total_price,event_id,channel
0,1,A87144476G,user400@rockets.com,77066,280-379-5220,109,9,1,200,3223,Web
1,2,A66578188Z,user141@rockets.com,76673,490-491-8071,101,10,4,800,3221,Box Office
2,3,A11689958W,user98@rockets.com,77031,244-805-9413,100,18,8,1600,3237,Box Office
3,4,A47432461Z,user213@rockets.com,76136,826-458-9773,400,7,1,50,3240,Web
4,5,A80089942I,user472@rockets.com,75559,803-733-6051,414,17,1,25,3215,Box Office
...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,A98035804M,user332@rockets.com,77616,524-512-1663,422,17,6,90,3244,Web
9996,9997,A62759828F,user146@rockets.com,76853,862-357-3734,400,15,8,400,3221,Web
9997,9998,A96104538T,user222@rockets.com,77086,840-386-8705,115,11,3,510,3236,Box Office
9998,9999,A38147058N,user495@rockets.com,76135,290-551-1299,403,18,7,350,3238,BackOffice


In [18]:
survey_data

,Submission ID,Attribute,Value
0,1,phone_no,290-551-1299
1,1,event_id,3220
2,1,how_satisfied_were_you_with_this_event,2
3,1,how_satisfied_were_you_with_your_retail_experi...,3
4,1,how_likely_are_you_to_attend_this_event_in_the...,5 - Very Likely
...,...,...,...
11995,1500,how_satisfied_were_you_with_your_retail_experi...,1 - Very Dissatisfied
11996,1500,how_likely_are_you_to_attend_this_event_in_the...,4
11997,1500,what_is_your_birthdate,16911
11998,1500,what_is_your_household_income,"$100,000 - $149,000"


**Takeaways**
 - Created acquire function that imports all data tables from files.